#AWS EC2 With Boto3  
EC2 = Elastic Compute Cloud

####Tasks:
1. Get EC2 Info
2. Create Key Pair and save the PEM file
3. Create Security Group and Inbound rule
4. Create EC2
5. Start/Stop/Reboot EC2
6. Terminate Instance
7. Clean up

Task 1: Getting information about existing EC2 instances

In [ ]:
import boto3
ec2 = boto3.client("ec2")
ec2.describe_instances()

Task 2: Create Key Pair and save the PEM file

In [ ]:
import boto3
ec2 = boto3.client("ec2")

#Delete if already exists
ec2.delete_key_pair(KeyName="ec2-boto3")

#Create a new key pair
keypair=ec2.create_key_pair(KeyName="ec2-boto3")

#Save to a file
file = open("ec2-boto3.pem","w")
file.write(keypair["KeyMaterial"])
file.close()

Task 3: Create Security Group and Inbound rule

In [ ]:
import boto3
ec2 = boto3.client("ec2")

# ec2.describe_security_groups()

#Delete if already exists
ec2.delete_security_group(GroupName="ec2-boto3-sg")

# create security group
sg = ec2.create_security_group(
    GroupName="ec2-boto3-sg",
    Description="ec2-boto3 security group",
    VpcId="vpc-02374f6cedca75b8a"
)

# get group id
groupid = sg["GroupId"]

# add ingress rule for http and ssh
ec2.authorize_security_group_ingress(
    GroupId=groupid,    
    IpPermissions=[{
        "IpProtocol":"tcp",
        "IpRanges": [{"CidrIp":"0.0.0.0/0"}],
        "FromPort":80,
        "ToPort":80
    },{
        "IpProtocol":"tcp",
        "IpRanges": [{"CidrIp":"0.0.0.0/0"}],
        "FromPort":22,
        "ToPort":22
    }]
)

Task 4: Create EC2 (with EBS volume)

In [ ]:
import boto3

#NEED to use Resource here, not client
ec2_res = boto3.resource("ec2")

#Create instance
instance = ec2_res.create_instances(
    ImageId="ami-0a4e637babb7b0a86",
    MinCount=1,
    MaxCount=1,
    InstanceType="t2.micro",
    KeyName="ec2-boto3",
    SecurityGroups=["ec2-boto3-sg"],
    BlockDeviceMappings=[
        {
            "DeviceName":"/dev/xvda",
            "Ebs":{
                "DeleteOnTermination": True,
                "VolumeSize":10
            }
        }
    ]
)

Task 5: Start/Stop EC2 Instances

In [ ]:
import boto3
ec2 = boto3.client("ec2")

#Stop instance
ec2.stop_instances(InstanceIds=["i-0925e872543eb83f8"])

#Start instance
ec2.start_instances(InstanceIds=["i-0925e872543eb83f8"])

Task 6: Terminate Instance

In [ ]:
import boto3
ec2 = boto3.client("ec2")

#Terminate instance
ec2.terminate_instances(InstanceIds=["i-0925e872543eb83f8"])

Task 7: Clean up

In [26]:
#Delete other stuff
ec2.delete_key_pair(KeyName="ec2-boto3")
ec2.delete_security_group(GroupName="ec2-boto3-sg")

{'ResponseMetadata': {'RequestId': 'f27e1d88-960b-44cb-8e06-ee47080d46cb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f27e1d88-960b-44cb-8e06-ee47080d46cb',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '239',
   'date': 'Mon, 28 Feb 2022 12:08:33 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}